In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pandas as pd

! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
PATH = 'bitcoin_preprocess.csv'

## Preprocessing

In [3]:
sp500 = pd.read_csv(PATH)

In [4]:
# Checking null
sp500.isnull().sum()

Unnamed: 0      0
time            0
open            0
high            0
low             0
close           0
Volume          0
Volume MA       0
MA              0
Histogram       0
MACD            0
Signal          0
RSI             0
RSI-based MA    0
dtype: int64

In [5]:
#  Checking nan
sp500.isna().sum()

Unnamed: 0      0
time            0
open            0
high            0
low             0
close           0
Volume          0
Volume MA       0
MA              0
Histogram       0
MACD            0
Signal          0
RSI             0
RSI-based MA    0
dtype: int64

In [6]:
#  Dropnan
sp500.dropna(inplace = True)
sp500.isna().any()

Unnamed: 0      False
time            False
open            False
high            False
low             False
close           False
Volume          False
Volume MA       False
MA              False
Histogram       False
MACD            False
Signal          False
RSI             False
RSI-based MA    False
dtype: bool

In [7]:
sp500['time'] = pd.to_datetime(sp500.time)

In [8]:
sp500.shape

(19975, 14)

### Data Exploration

In [9]:
print("Starting date: ",sp500.iloc[0][0])
print("Ending date: ", sp500.iloc[-1][0])
print("Duration: ", sp500.iloc[-1][0]-sp500.iloc[0][0])

Starting date:  88
Ending date:  20062
Duration:  19974


In [10]:
# sp500.to_csv('bitcoin_preprocess.csv')

## Machine Learning

In [11]:
from utils.utils import create_data_train, read_csv
from configs.config import *
from utils.preprocess import Preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 

In [12]:
path = 'bitcoin_preprocess.csv'
ts_config = TSConfig()
frame = read_csv(path, ts_config.train_features, ts_config.pred_features, return_np = True)
preprocess = Preprocessing()
ML = True

bitcoin_preprocess.csv


In [13]:
feature = frame['feature']
target = frame['target']

In [14]:
target.shape

(19975, 2)

In [15]:
#   Scaling data
feature, (feature_max_scale, feature_min_scale) = preprocess.apply_minmax_scale(feature)

In [16]:
target, (target_max_scale, target_min_scale) = preprocess.apply_minmax_scale(target)

In [17]:
train_test_ratio = 0.8
seq_steps        = 7

In [18]:
X, Y = create_data_train(feature, target, seq_steps)
if ML:
    X = X.detach().cpu().numpy()
    Y = Y.detach().cpu().numpy()

In [19]:
split_idx = int(X.shape[0] * train_test_ratio)


In [20]:
#   Predict high
x_train, y_train = X[:split_idx, :, :], Y[:split_idx, :]
x_test, y_test   = X[split_idx:, :, :], Y[split_idx:, :]

print(y_train.shape, y_test.shape)

(15971, 2) (3993, 2)


In [21]:
print(f'Number of traning samples = {x_train.shape[0]}\nNumber of testing samples = {y_test.shape[0]}')
print(x_train.shape)

Number of traning samples = 15971
Number of testing samples = 3993
(15971, 7, 10)


In [23]:
def max_min_inverse(inp, max_scale, min_scale):
    print(max_scale)
    return (inp * (max_scale - min_scale)) + min_scale

In [25]:
#   Predict categories
"""
'high': 0
'low': 1
'MA:   2
'MACD': 3
'open': 4
'close': 5
'Volume': 6
'Volume MA': 7
'MA': 8
'Histogram': 9
'MACD': 10
'Signal': 11
"""
category = 1
x_train_ml = x_train[:, :, category]
x_test_ml = x_test[:, :, category]
y_train_ml = y_train[:, category]
y_test_ml  = y_test[:,category]

print(f'Feature selection = {ts_config.pred_features[category]}')
print(f'Train shape: {x_train_ml.shape}, {y_train_ml.shape}\nTest shape: {x_test_ml.shape}, {y_test_ml.shape}')

Feature selection = low
Train shape: (15971, 7), (15971,)
Test shape: (3993, 7), (3993,)


### Support Vector Regressor

In [26]:
from sklearn.svm import SVR
from configs.config import BASEDIR
import pickle

In [27]:
def evaluate_SVR(x_train, y_train, x_test, y_test, C, gamma, category):
    svr_rbf = SVR(kernel = 'rbf', C= 1e2, gamma= 0.1)
    svr_rbf.fit(x_train, y_train)

    train_predict=svr_rbf.predict(x_train)
    test_predict=svr_rbf.predict(x_test)

    y_test_rescale = max_min_inverse(y_test, target_max_scale[:, category], target_min_scale[:, category])
    pred_rescale   = max_min_inverse(test_predict, target_max_scale[:, category], target_min_scale[:, category])

    train_predict = train_predict.reshape(-1,1)
    test_predict = test_predict.reshape(-1,1)
    
    mean_sqr_err = mean_squared_error(y_test_rescale, pred_rescale)
    mean_abs_err = mean_absolute_error(y_test_rescale, pred_rescale)
    r_square     = r2_score(y_test_rescale, pred_rescale)
    print("Train data prediction:", train_predict.shape)
    print("Test data prediction:", test_predict.shape)
    print(f'Prediction of {ts_config.pred_features[category]}')
    print(f'MAE = {mean_abs_err}\nMSE = {mean_sqr_err}\nR square = {r_square}')

    results = {
        'prediction_' + ts_config.pred_features[category]: pred_rescale,
        'ground_truth': y_test_rescale,
        'MSE': mean_sqr_err,
        'MAE': mean_abs_err,
        'R2': r_square,
        'RMSE': np.sqrt(mean_sqr_err)
    }

    print(results)
    save_pkl_path = BASEDIR / f'save/SVR_{ts_config.pred_features[category]}.pkl'
    with open(save_pkl_path, mode=  'wb') as writer:
        pickle.dump(results, writer)
    writer.close()

In [28]:
evaluate_SVR(x_train_ml, y_train_ml, x_test_ml, y_test_ml, 1e2, 1e-1, category)

[68451.19]
[68451.19]
Train data prediction: (15971, 1)
Test data prediction: (3993, 1)
Prediction of low
MAE = 3405.600834303915
MSE = 15943935.772305543
R square = 0.14403223440365165
{'prediction_low': array([24057.71610473, 24038.09607766, 24071.82078595, ...,
       29445.67930355, 29443.359772  , 29463.08868224]), 'ground_truth': array([20407.66003502, 20500.02993242, 20457.61985553, ...,
       29801.00067141, 29845.95022187, 29764.27974154]), 'MSE': 15943935.772305543, 'MAE': 3405.600834303915, 'R2': 0.14403223440365165, 'RMSE': 3992.9858217010415}


### Random Forest

In [29]:
from sklearn.ensemble import RandomForestRegressor
import pickle

In [30]:
def evaluate_randomfr(x_train, y_train, x_test, y_test, C, gamma, category):


    regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
    regressor.fit(x_train, y_train)

    train_predict=regressor.predict(x_train)
    test_predict=regressor.predict(x_test)

    y_test_rescale = max_min_inverse(y_test, target_max_scale[:, category], target_min_scale[:, category])
    pred_rescale   = max_min_inverse(test_predict, target_max_scale[:, category], target_min_scale[:, category])

    train_predict = train_predict.reshape(-1,1)
    test_predict = test_predict.reshape(-1,1)
    
    mean_sqr_err = mean_squared_error(y_test_rescale, pred_rescale)
    mean_abs_err = mean_absolute_error(y_test_rescale, pred_rescale)
    r_square     = r2_score(y_test_rescale, pred_rescale)
    print("Train data prediction:", train_predict.shape)
    print("Test data prediction:", test_predict.shape)
    print(f'Prediction of {ts_config.pred_features[category]}')
    print(f'MAE = {mean_abs_err}\nMSE = {mean_sqr_err}\nR square = {r_square}')

    results = {
        'prediction_' + ts_config.pred_features[category]: pred_rescale,
        'ground_truth': y_test_rescale,
        'MSE': mean_sqr_err,
        'MAE': mean_abs_err,
        'R2': r_square,
        'RMSE': np.sqrt(mean_sqr_err)
    }

    print(results)
    save_pkl_path = BASEDIR / f'save/RandomF/rfr_{ts_config.pred_features[category]}.pkl'
    with open(save_pkl_path, mode=  'wb') as writer:
        pickle.dump(results, writer)
    writer.close()

In [31]:
evaluate_randomfr(x_train_ml, y_train_ml, x_test_ml, y_test_ml, 1e2, 1e-1, category)

[68451.19]
[68451.19]
Train data prediction: (15971, 1)
Test data prediction: (3993, 1)
Prediction of low
MAE = 610.1955784446704
MSE = 815166.9837671482
R square = 0.9562368619864189
{'prediction_low': array([20466.50388844, 20420.23739556, 20508.44227213, ...,
       30101.81842153, 29720.85313319, 29957.07138054]), 'ground_truth': array([20407.66003502, 20500.02993242, 20457.61985553, ...,
       29801.00067141, 29845.95022187, 29764.27974154]), 'MSE': 815166.9837671482, 'MAE': 610.1955784446704, 'R2': 0.9562368619864189, 'RMSE': 902.8659832816542}


### ARIMA

In [3]:
from statsmodels.tsa.arima_model import ARIMA

## Stacked LSTM

In [24]:
import tensorflow as tf 
from tensorflow.keras.layers import LSTM, Dense, GRU
from tensorflow.keras import Sequential

In [25]:
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(16,return_sequences=True,input_shape=(ts_config.n_steps, ts_config.n_features)))
model.add(LSTM(16))
model.add(Dense(ts_config.n_classes))
model.compile(loss='mean_squared_error',optimizer='adam')

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 7, 16)             1728      
                                                                 
 lstm_1 (LSTM)               (None, 16)                2112      
                                                                 
 dense (Dense)               (None, 2)                 34        
                                                                 
Total params: 3,874
Trainable params: 3,874
Non-trainable params: 0
_________________________________________________________________


In [27]:
x_train_dl = X[:split_idx, :, :]
y_train_dl = Y[:split_idx, :]
x_test_dl  = X[split_idx: , :, :]
y_test_dl  = Y[split_idx:, :]

print(f'Train samples: {x_train_dl.shape} -- Test samples: {x_test_dl.shape}')
print(f'Train targets: {y_train_dl.shape} -- Test targets: {y_test_dl.shape}')

Train samples: (15971, 7, 10) -- Test samples: (3993, 7, 10)
Train targets: (15971, 2) -- Test targets: (3993, 2)


In [28]:
model.fit(x_train_dl,y_train_dl,validation_data=(x_test_dl,y_test_dl),epochs=100,batch_size=512,verbose=1)
model.save(f'save/StackedLSTM/ckpt/lstm.h5')

Epoch 1/100
32/32 [==============================] - 3s 26ms/step - loss: 0.0844 - val_loss: 0.0581
Epoch 2/100
32/32 [==============================] - 1s 17ms/step - loss: 0.0122 - val_loss: 0.0077
Epoch 3/100
32/32 [==============================] - 0s 13ms/step - loss: 0.0020 - val_loss: 0.0022
Epoch 4/100
32/32 [==============================] - 0s 11ms/step - loss: 9.8274e-04 - val_loss: 0.0017
Epoch 5/100
32/32 [==============================] - 0s 9ms/step - loss: 8.1238e-04 - val_loss: 0.0015
Epoch 6/100
32/32 [==============================] - 0s 11ms/step - loss: 6.9193e-04 - val_loss: 0.0014
Epoch 7/100
32/32 [==============================] - 0s 13ms/step - loss: 5.9216e-04 - val_loss: 0.0012
Epoch 8/100
32/32 [==============================] - 0s 12ms/step - loss: 5.1058e-04 - val_loss: 0.0011
Epoch 9/100
32/32 [==============================] - 0s 12ms/step - loss: 4.4406e-04 - val_loss: 0.0010
Epoch 10/100
32/32 [==============================] - 1s 16ms/step - loss: 3.

In [29]:
train_predict=model.predict(x_train_dl)
test_predict=model.predict(x_test_dl)
train_predict.shape, test_predict.shape

125/125 [==============================] - 0s 3ms/step


((15971, 2), (3993, 2))

In [30]:
test_pred_scale = (test_predict * (target_max_scale - target_min_scale)) + target_min_scale

In [31]:
y_test_scale = (y_test_dl * (target_max_scale - target_min_scale) + target_min_scale)

In [32]:
mean_squared_error(y_test_scale, test_pred_scale)

73265.8854507157

In [33]:
mean_absolute_error(y_test_scale, test_pred_scale)

194.57151302991824

In [34]:
r2_score(y_test_scale, test_pred_scale)

0.9960862344572969

In [35]:
import pickle

In [36]:
n_samples, n_fea = y_test_scale.shape
lstm_res = dict()
for idx, category in enumerate(ts_config.pred_features):
    cate_mse = mean_squared_error(y_test_scale[:, idx], test_pred_scale[:, idx])
    cate_mae = mean_absolute_error(y_test_scale[:, idx], test_pred_scale[:, idx])
    cate_rsqr = r2_score(y_test_scale[:, idx], test_pred_scale[:, idx])

    lstm_res[ts_config.pred_features[idx]] = [cate_mse, cate_mae, cate_rsqr]

    print(f'MSE in {ts_config.pred_features[idx]} = {cate_mse}')
    print(f'MAE in {ts_config.pred_features[idx]} = {cate_mae}')
    print(f'R2 score in {ts_config.pred_features[idx]} = {cate_rsqr}')
print(lstm_res)
with open(f'save/StackedLSTM/results.pkl', mode= 'wb') as writer:
    pickle.dump(lstm_res, writer)
writer.close()


MSE in high = 38938.329151169004
MAE in high = 134.84057755247565
R2 score in high = 0.9979487414261631
MSE in low = 107593.4417502624
MAE in low = 254.30244850736145
R2 score in low = 0.9942237274884308
{'high': [38938.329151169004, 134.84057755247565, 0.9979487414261631], 'low': [107593.4417502624, 254.30244850736145, 0.9942237274884308]}


In [37]:
y_test_scale.shape

(3993, 2)

In [38]:
test_pred_scale.shape

(3993, 2)

In [39]:
n_samples, n_fea = y_test_scale.shape

In [40]:
y_test_scale[:, 1].reshape((n_samples, 1)).shape

(3993, 1)

## Stacked GRU

In [41]:
tf.keras.backend.clear_session()
gru_model=Sequential()
gru_model.add(GRU(16,return_sequences=True,input_shape=(ts_config.n_steps, ts_config.n_features)))
gru_model.add(GRU(16))
gru_model.add(Dense(ts_config.n_classes))
gru_model.compile(loss='mean_squared_error',optimizer='adam')

In [42]:
gru_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 7, 16)             1344      
                                                                 
 gru_1 (GRU)                 (None, 16)                1632      
                                                                 
 dense (Dense)               (None, 2)                 34        
                                                                 
Total params: 3,010
Trainable params: 3,010
Non-trainable params: 0
_________________________________________________________________


In [44]:
gru_model.fit(x_train_dl,y_train_dl,validation_data=(x_test_dl,y_test_dl),epochs=100,batch_size=512,verbose=1)
gru_model.save(f'save/GRU/gru_model.h5')

Epoch 1/100
32/32 [==============================] - 0s 11ms/step - loss: 0.0030 - val_loss: 0.0031
Epoch 2/100
32/32 [==============================] - 0s 9ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 3/100
32/32 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 7.4371e-04
Epoch 4/100
32/32 [==============================] - 0s 10ms/step - loss: 9.0745e-04 - val_loss: 4.4754e-04
Epoch 5/100
32/32 [==============================] - 0s 10ms/step - loss: 7.2685e-04 - val_loss: 2.8358e-04
Epoch 6/100
32/32 [==============================] - 0s 10ms/step - loss: 5.9693e-04 - val_loss: 1.9178e-04
Epoch 7/100
32/32 [==============================] - 0s 7ms/step - loss: 4.9729e-04 - val_loss: 1.5193e-04
Epoch 8/100
32/32 [==============================] - 0s 10ms/step - loss: 4.1971e-04 - val_loss: 1.3644e-04
Epoch 9/100
32/32 [==============================] - 0s 10ms/step - loss: 3.5532e-04 - val_loss: 1.2611e-04
Epoch 10/100
32/32 [==============================

In [45]:
train_predict_gru = gru_model.predict(x_train_dl)
test_predict_gru = gru_model.predict(x_test_dl)
print(train_predict_gru.shape, test_predict_gru.shape)

125/125 [==============================] - 0s 2ms/step
(15971, 2) (3993, 2)


In [46]:
gru_test_scale = (test_predict_gru * (target_max_scale - target_min_scale)) + target_min_scale

In [47]:
gru_test_scale

array([[20543.21555121, 20498.90504904],
       [20605.19652628, 20512.10486479],
       [20604.16101809, 20550.97491395],
       ...,
       [29949.69418436, 29747.07099931],
       [29953.22070247, 29739.56072529],
       [29945.31419492, 29748.66399206]])

In [48]:

n_samples, n_fea = y_test_scale.shape
gru_res = dict()
for idx, category in enumerate(ts_config.pred_features):
    cate_mse = mean_squared_error(y_test_scale[:, idx], gru_test_scale[:, idx])
    cate_mae = mean_absolute_error(y_test_scale[:, idx], gru_test_scale[:, idx])
    cate_rsqr = r2_score(y_test_scale[:, idx], gru_test_scale[:, idx])

    gru_res[ts_config.pred_features[idx]] = [cate_mse, cate_mae, cate_rsqr]

    print(f'MSE in {ts_config.pred_features[idx]} = {cate_mse}')
    print(f'MAE in {ts_config.pred_features[idx]} = {cate_mae}')
    print(f'R2 score in {ts_config.pred_features[idx]} = {cate_rsqr}')
print(gru_res)
with open(f'save/GRU/results_gru.pkl', mode= 'wb') as writer:
    pickle.dump(gru_res, writer)
writer.close()


MSE in high = 30038.733093490624
MAE in high = 129.368841496856
R2 score in high = 0.9984175692653373
MSE in low = 38123.07366028278
MAE in low = 152.55943590016585
R2 score in low = 0.9979533207706883
{'high': [30038.733093490624, 129.368841496856, 0.9984175692653373], 'low': [38123.07366028278, 152.55943590016585, 0.9979533207706883]}


In [49]:
gru_test_scale

array([[20543.21555121, 20498.90504904],
       [20605.19652628, 20512.10486479],
       [20604.16101809, 20550.97491395],
       ...,
       [29949.69418436, 29747.07099931],
       [29953.22070247, 29739.56072529],
       [29945.31419492, 29748.66399206]])

In [50]:
y_test_scale

array([[20540.69995243, 20407.66003502],
       [20536.37984876, 20500.02993242],
       [20531.37006729, 20457.61985553],
       ...,
       [29919.15006391, 29801.00067141],
       [29922.50049408, 29845.95022187],
       [29888.88038684, 29764.27974154]])

## Our method

In [51]:
# from model import TSModel
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras import Sequential

ModuleNotFoundError: No module named 'pytorch_lightning'

In [29]:
tf.keras.backend.clear_session()
bidir_model=Sequential()
bidir_model.add(Bidirectional(LSTM(16,return_sequences=True,input_shape=(ts_config.n_steps, ts_config.n_features))))
bidir_model.add(Bidirectional(LSTM(16)))
bidir_model.add(Dense(ts_config.n_classes))
bidir_model.compile(loss='mean_squared_error',optimizer='adam')

In [32]:
# bidir_model.summary()
bidir_model.fit(x_train_dl,y_train_dl,validation_data=(x_test_dl,y_test_dl),epochs=100,batch_size=512,verbose=1)

Epoch 1/100
33/33 [==============================] - 9s 56ms/step - loss: 0.0309 - val_loss: 0.0914
Epoch 2/100
33/33 [==============================] - 1s 18ms/step - loss: 0.0019 - val_loss: 0.0137
Epoch 3/100
33/33 [==============================] - 1s 20ms/step - loss: 2.4472e-04 - val_loss: 0.0143
Epoch 4/100
33/33 [==============================] - 1s 17ms/step - loss: 1.5637e-04 - val_loss: 0.0124
Epoch 5/100
33/33 [==============================] - 1s 17ms/step - loss: 1.3129e-04 - val_loss: 0.0115
Epoch 6/100
33/33 [==============================] - 1s 16ms/step - loss: 1.1312e-04 - val_loss: 0.0109
Epoch 7/100
33/33 [==============================] - 1s 17ms/step - loss: 9.9040e-05 - val_loss: 0.0103
Epoch 8/100
33/33 [==============================] - 1s 17ms/step - loss: 8.8116e-05 - val_loss: 0.0098
Epoch 9/100
33/33 [==============================] - 1s 17ms/step - loss: 7.9435e-05 - val_loss: 0.0092
Epoch 10/100
33/33 [==============================] - 1s 19ms/step - los

In [68]:
train_predict_bidir = bidir_model.predict(x_train_dl)
test_predict_bidir = bidir_model.predict(x_test_dl)
print(train_predict_bidir.shape, test_predict_bidir.shape)
bidir_model.save(f'save/BiLSTM_Time/model.h5')

130/130 [==============================] - 0s 4ms/step
(16587, 5) (4147, 5)


In [69]:
bidir_pred_scale = (test_predict_bidir * (target_max_scale - target_min_scale)) + target_min_scale

In [70]:
y_test_scale

array([[3573.65007628, 3556.85003304, 3562.29623436, 3563.62002571,
        3557.53995056],
       [3589.81011779, 3564.09003568, 3563.2345051 , 3566.82004822,
        3564.67999336],
       [3575.89992754, 3556.54997781, 3564.23918094, 3564.68004395,
        3559.79999797],
       ...,
       [4105.14005328, 4086.87000917, 4001.32070697, 4103.03988975,
        4096.89007598],
       [4105.23998636, 4093.1898917 , 4002.93473328, 4096.87995119,
        4104.00002428],
       [4106.62991969, 4093.55005652, 4004.43386442, 4104.02003685,
        4093.72004005]])

In [75]:

n_samples, n_fea = y_test_scale.shape
bidir_result = dict()
for idx, category in enumerate(ts_config.pred_features):
    cate_mse = mean_squared_error(y_test_scale[:, idx], bidir_pred_scale[:, idx])
    cate_mae = mean_absolute_error(y_test_scale[:, idx], bidir_pred_scale[:, idx])
    cate_rsqr = r2_score(y_test_scale[:, idx], bidir_pred_scale[:, idx])

    bidir_result[ts_config.pred_features[idx]] = [cate_mse, cate_mae, cate_rsqr]

    print(f'MSE in {ts_config.pred_features[idx]} = {cate_mse}')
    print(f'MAE in {ts_config.pred_features[idx]} = {cate_mae}')
    print(f'R2 score in {ts_config.pred_features[idx]} = {cate_rsqr}')
print(bidir_result)
with open(f'save/BiLSTM_Time/results_bidir.pkl', mode= 'wb') as writer:
    pickle.dump(bidir_result, writer)
writer.close()


MSE in high = 1786.8458572651514
MAE in high = 31.507823106917467
R2 score in high = 0.9803716699938331
MSE in low = 5312.98377266897
MAE in low = 55.781296917008355
R2 score in low = 0.9421939077249079
MSE in MA = 108465.93359541956
MAE in MA = 278.58631885526825
R2 score in MA = -0.19983390964606573
MSE in open = 13356.648659201868
MAE in open = 93.2332664953878
R2 score in open = 0.8540694684644325
MSE in close = 16775.75641565107
MAE in close = 108.98252168946306
R2 score in close = 0.8165802970081102
{'high': [1786.8458572651514, 31.507823106917467, 0.9803716699938331], 'low': [5312.98377266897, 55.781296917008355, 0.9421939077249079], 'MA': [108465.93359541956, 278.58631885526825, -0.19983390964606573], 'open': [13356.648659201868, 93.2332664953878, 0.8540694684644325], 'close': [16775.75641565107, 108.98252168946306, 0.8165802970081102]}
